<a href="https://colab.research.google.com/github/Excalib3r/pinecone/blob/main/ph5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install langchain

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:

pip install pinecone-client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.1/179.1 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 19.0 MB/s eta 0:00:00


In [ ]:
# Make the display a bit wider
# from IPython.display import display, HTML
# display(HTML("<style>.container { width:90% !important; }</style>"))

# LangChain basics
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain
from langchain.chains import create_extraction_chain

# Vector Store and retrievals
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import Pinecone
import pinecone

# Chat Prompt templates for dynamic values
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)

/usr/local/lib/python3.10/dist-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [ ]:
pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 3.7 MB/s eta 0:00:00


In [ ]:
OPENAI_API_KEY = ''

In [ ]:
# Creating two versions of the model so I can swap between gpt3.5 and gpt4
llm3 = ChatOpenAI(temperature=0,
                  openai_api_key=OPENAI_API_KEY,
                  model_name="gpt-3.5-turbo-0613",
                  request_timeout = 180
                )

llm4 = ChatOpenAI(temperature=0,
                  openai_api_key=OPENAI_API_KEY,
                  model_name="gpt-4-0613",
                  request_timeout = 180
                 )

In [ ]:
# I put three prepared transcripts
# transcript_paths = [
#     './Sunrise_Employee_Handbook_Jan2015.txt',
# ]

with open('./user-qna.txt') as file:
    transcript = file.read()

In [ ]:
print (len(transcript))

5112


In [ ]:
# Load up your text splitter
text_splitter = RecursiveCharacterTextSplitter(separators=["\n\n", "\n", " "], chunk_size=2000, chunk_overlap=200)



In [ ]:
# Load up your text splitter
text_splitter = RecursiveCharacterTextSplitter(separators=["\n\n", "\n", " "], chunk_size=10000, chunk_overlap=2200)

# I'm only doing the first 23250 characters. This to save on costs. When you're doing your exercise you can remove this to let all the data through
transcript_subsection_characters = 23250

In [ ]:
pip install tiktoken

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 27.4 MB/s eta 0:00:00


In [ ]:
docs = text_splitter.create_documents([transcript[:transcript_subsection_characters]])
print (f"You have {len(docs)} docs. First doc is {llm3.get_num_tokens(docs[0].page_content)} tokens")

You have 1 docs. First doc is 1023 tokens


In [ ]:
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [ ]:
PINECONE_API_KEY = ''
PINECONE_API_ENV = ''

In [ ]:
pinecone.init(
    api_key=PINECONE_API_KEY,  # find at app.pinecone.io
    environment=PINECONE_API_ENV  # next to api key in console
)

In [ ]:
index_name = ""

docsearch = Pinecone.from_documents(docs, embeddings, index_name=index_name)

In [ ]:
# The system instructions. Notice the 'context' placeholder down below. This is where our relevant docs will go.
# The 'question' in the human message below won't be a question per se, but rather a topic we want to get relevant information on
system_template = """
You will be given text from a employee handbook contains many topics.

You will be given text that contains an sophisticated AI's judgement of the users answers.
Use the explanation and missing point from each question to summarize a conclusion of what the user needs to learn.
Use the context given, dont go outside the context for your answers.
Do not respond with information that isn't relevant to the helping the user learn what he doesnt know.
----------------
{context}"""

messages = [
    SystemMessagePromptTemplate.from_template(system_template),
    HumanMessagePromptTemplate.from_template("{question}"),
]

# This will pull the two messages together and get them ready to be sent to the LLM through the retriever
CHAT_PROMPT = ChatPromptTemplate.from_messages(messages)

In [ ]:
# I'm using gpt4 for the increased reasoning power.
# I'm also setting k=4 so the number of relevant docs we get back is 4. This parameter should be tuned to your use case
qa = RetrievalQA.from_chain_type(llm=llm3,
                                 chain_type="stuff",
                                 retriever=docsearch.as_retriever(k=3),
                                 chain_type_kwargs = {
#                                      'verbose': True,
                                     'prompt': CHAT_PROMPT
                                 })

In [ ]:


expanded_doc = qa.run(docsearch)

# Print out the query and the results of the operation.

print(expanded_doc)
print ("\n\n")

TypeError: ignored

In [ ]:
template="""

You will be given text from a employee handbook contains many topics.

You will be given text that contains an sophisticated AI's judgement of the users answers.
Use the explanation and missing point from each question to summarize a conclusion of what the user needs to learn.
Use the context given, dont go outside the context for your answers.
Do not respond with information that isn't relevant to the helping the user learn what he doesnt know.
List it in bullet points
----------------
"""


system_message_prompt_map = SystemMessagePromptTemplate.from_template(template)

human_template="Transcript: {text}" # Simply just pass the text as a human message
human_message_prompt_map = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt_map = ChatPromptTemplate.from_messages(messages=[system_message_prompt_map, human_message_prompt_map])

In [ ]:
chain = load_summarize_chain(llm3,
                             chain_type="map_reduce",
                             map_prompt=chat_prompt_map,

#                              verbose=True
                            )

In [ ]:
topics_found = chain.run({"input_documents": docs})

In [ ]:
print (topics_found)

The compensability of on-call time depends on factors such as location and restrictions on personal use. On-call time is typically for non-exempt employees eligible for overtime pay. Travel time for on-call employees is generally not compensated, but there are exceptions. The compensability of time spent working or responding to calls depends on employee classification and employer policies. Specific circumstances and exemptions should be considered when determining the compensability of on-call time.


In [ ]:
expanded_topic = qa.run(query)

NameError: ignored